In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
file = 'data/Hz31.txt'
data = pd.read_csv(file, names=['z', 'hz', 'err'], sep = " " )
# besthzorder = np.array([25,  3,  9,  7, 19,  2, 17, 16,  6, 28, 12, 20, 21, 26, 27, 11,  1,
#        10, 30, 13, 29,  0,  8, 18, 24,  5, 23,  4, 22, 15, 14])
# data = data.values[a]

# file = 'data/fs8Diagram.txt'
# data = pd.read_csv(file, names=['z', '$f\sigma_8$', 'err', '$\Omega_ref$'], sep = "\t", skiprows=3)
# data = data[['z', '$f\sigma_8$', 'err']]
# randomize = np.random.permutation(len(data.values))
# a = np.array([21,1,3,10,12,17,8,4,7,16, 2,11,19,13,18,5,9,20,15,14,6,0])
even = [x for x in range(len(data.values)) if x%2 ==0]
odd = [x for x in range(len(data.values)) if x%2 ==1]
even.extend(odd)
order = np.array(even)

data = data.values[order]


X = data[:,0]
y = data[:,1:]

scalerx = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerx.fit(X.reshape(-1,1))
# apply transform
X = scalerx.transform(X.reshape(-1,1))


# scalery = MinMaxScaler(feature_range=(0,1))
# scalery = StandardScaler()
# scalery.fit(y)
# y = scalery.transform(y)

split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)


In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=50,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [5]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8, 16, 32, 64]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def hzmodel(input_z, hparams):
    #encoder
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    ehidden1 = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='relu')(ehidden2)
        
    return elast

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks, verbose=0) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [12]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                  HP_NUM_UNITS1: num_units1,
                  HP_NUM_UNITS2: num_units2,
                  HP_NUM_UNITS3: num_units3,
                #                       HP_DROPOUT: dropout_rate,
#                       HP_OPTIMIZER: optimizer,
                  HP_BATCHSIZE: batch_size
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
#                 print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
1/1 [==============================] - 0s 931us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-1
1/1 [==============================] - 0s 812us/step - loss: 378.7719 - mean_squared_error: 378.7719
--- Starting trial: run-2
1/1 [==============================] - 0s 811us/step - loss: 683.6348 - mean_squared_error: 683.6348
--- Starting trial: run-3
1/1 [==============================] - 0s 1ms/step - loss: 362.9297 - mean_squared_error: 362.9297
--- Starting trial: run-4
1/1 [==============================] - 0s 1ms/step - loss: 10710.2520 - mean_squared_error: 10710.2520
--- Starting trial: run-5
1/1 [==============================] - 0s 803us/step - loss: 350.3932 - mean_squared_error: 350.3932
--- Starting trial: run-6
1/1 [==============================] - 0s 770us/step - loss: 401.9655 - mean_squared_error: 401.9655
--- Starting trial: run-7
1/1 [==============================] - 0s 779us/step - loss: 11034.5000 - mean_sq

1/1 [==============================] - 0s 726us/step - loss: 372.5813 - mean_squared_error: 372.5813
--- Starting trial: run-13
1/1 [==============================] - 0s 827us/step - loss: 369.2790 - mean_squared_error: 369.2790
--- Starting trial: run-14
1/1 [==============================] - 0s 771us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-15
1/1 [==============================] - 0s 1ms/step - loss: 352.6956 - mean_squared_error: 352.6956
--- Starting trial: run-16
1/1 [==============================] - 0s 1ms/step - loss: 358.5504 - mean_squared_error: 358.5504
--- Starting trial: run-17
1/1 [==============================] - 0s 742us/step - loss: 340.3713 - mean_squared_error: 340.3713
--- Starting trial: run-18
1/1 [==============================] - 0s 1ms/step - loss: 342.3681 - mean_squared_error: 342.3681
--- Starting trial: run-19
1/1 [==============================] - 0s 842us/step - loss: 354.5103 - mean_squared_error: 354.5103
--- S

1/1 [==============================] - 0s 765us/step - loss: 368.8340 - mean_squared_error: 368.8340
--- Starting trial: run-23
1/1 [==============================] - 0s 908us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-24
1/1 [==============================] - 0s 1ms/step - loss: 361.0901 - mean_squared_error: 361.0901
--- Starting trial: run-25
1/1 [==============================] - 0s 825us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-26
1/1 [==============================] - 0s 785us/step - loss: 336.6685 - mean_squared_error: 336.6685
--- Starting trial: run-27
1/1 [==============================] - 0s 1ms/step - loss: 10710.8389 - mean_squared_error: 10710.8389
--- Starting trial: run-28
1/1 [==============================] - 0s 779us/step - loss: 368.5572 - mean_squared_error: 368.5572
--- Starting trial: run-29
1/1 [==============================] - 0s 875us/step - loss: 343.6997 - mean_squared_error: 343.

1/1 [==============================] - 0s 867us/step - loss: 394.8160 - mean_squared_error: 394.8160
--- Starting trial: run-33
1/1 [==============================] - 0s 1ms/step - loss: 348.5172 - mean_squared_error: 348.5172
--- Starting trial: run-34
1/1 [==============================] - 0s 857us/step - loss: 359.1160 - mean_squared_error: 359.1160
--- Starting trial: run-35
1/1 [==============================] - 0s 2ms/step - loss: 657.4764 - mean_squared_error: 657.4764
--- Starting trial: run-36
1/1 [==============================] - 0s 921us/step - loss: 366.0134 - mean_squared_error: 366.0134
--- Starting trial: run-37
1/1 [==============================] - 0s 927us/step - loss: 351.9328 - mean_squared_error: 351.9328
--- Starting trial: run-38
1/1 [==============================] - 0s 1ms/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-39
1/1 [==============================] - 0s 1ms/step - loss: 383.7585 - mean_squared_error: 383.7585
--- Sta

1/1 [==============================] - 0s 1ms/step - loss: 356.1629 - mean_squared_error: 356.1629
--- Starting trial: run-43
1/1 [==============================] - 0s 848us/step - loss: 387.5202 - mean_squared_error: 387.5202
--- Starting trial: run-44
1/1 [==============================] - 0s 761us/step - loss: 393.1425 - mean_squared_error: 393.1425
--- Starting trial: run-45
1/1 [==============================] - 0s 1ms/step - loss: 380.4100 - mean_squared_error: 380.4100
--- Starting trial: run-46
1/1 [==============================] - 0s 1ms/step - loss: 357.0670 - mean_squared_error: 357.0670
--- Starting trial: run-47
1/1 [==============================] - 0s 1ms/step - loss: 360.7668 - mean_squared_error: 360.7668
--- Starting trial: run-48
1/1 [==============================] - 0s 801us/step - loss: 654.8558 - mean_squared_error: 654.8558
--- Starting trial: run-49
1/1 [==============================] - 0s 858us/step - loss: 356.3881 - mean_squared_error: 356.3881
--- Startin

...

1/1 [==============================] - 0s 936us/step - loss: 397.1650 - mean_squared_error: 397.1650
--- Starting trial: run-273
1/1 [==============================] - 0s 1ms/step - loss: 390.4275 - mean_squared_error: 390.4275
--- Starting trial: run-274
1/1 [==============================] - 0s 879us/step - loss: 388.8961 - mean_squared_error: 388.8961
--- Starting trial: run-275
1/1 [==============================] - 0s 742us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-276
1/1 [==============================] - 0s 905us/step - loss: 10715.4707 - mean_squared_error: 10715.4707
--- Starting trial: run-277
1/1 [==============================] - 0s 1ms/step - loss: 388.0976 - mean_squared_error: 388.0976
--- Starting trial: run-278
1/1 [==============================] - 0s 910us/step - loss: 366.8570 - mean_squared_error: 366.8570
--- Starting trial: run-279
1/1 [==============================] - 0s 840us/step - loss: 369.5587 - mean_squared_error: 3

1/1 [==============================] - 0s 1ms/step - loss: 10712.5488 - mean_squared_error: 10712.5488
--- Starting trial: run-283
1/1 [==============================] - 0s 821us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-284
1/1 [==============================] - 0s 742us/step - loss: 385.5204 - mean_squared_error: 385.5204
--- Starting trial: run-285
1/1 [==============================] - 0s 936us/step - loss: 368.4627 - mean_squared_error: 368.4627
--- Starting trial: run-286
1/1 [==============================] - 0s 654us/step - loss: 361.4178 - mean_squared_error: 361.4178
--- Starting trial: run-287
1/1 [==============================] - 0s 1ms/step - loss: 355.4967 - mean_squared_error: 355.4967
--- Starting trial: run-288
1/1 [==============================] - 0s 798us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-289
1/1 [==============================] - 0s 1ms/step - loss: 11034.5000 - mean_squared_erro

1/1 [==============================] - 0s 1ms/step - loss: 10712.3584 - mean_squared_error: 10712.3584
--- Starting trial: run-293
1/1 [==============================] - 0s 924us/step - loss: 364.9879 - mean_squared_error: 364.9879
--- Starting trial: run-294
1/1 [==============================] - 0s 836us/step - loss: 371.9170 - mean_squared_error: 371.9170
--- Starting trial: run-295
1/1 [==============================] - 0s 1ms/step - loss: 364.3716 - mean_squared_error: 364.3716
--- Starting trial: run-296
1/1 [==============================] - 0s 869us/step - loss: 372.5779 - mean_squared_error: 372.5779
--- Starting trial: run-297
1/1 [==============================] - 0s 889us/step - loss: 372.0125 - mean_squared_error: 372.0125
--- Starting trial: run-298
1/1 [==============================] - 0s 885us/step - loss: 686.4139 - mean_squared_error: 686.4139
--- Starting trial: run-299
1/1 [==============================] - 0s 630us/step - loss: 11034.5000 - mean_squared_error: 110

1/1 [==============================] - 0s 1ms/step - loss: 362.8713 - mean_squared_error: 362.8713
--- Starting trial: run-303
1/1 [==============================] - 0s 1ms/step - loss: 684.6476 - mean_squared_error: 684.6476
--- Starting trial: run-304
1/1 [==============================] - 0s 962us/step - loss: 385.2839 - mean_squared_error: 385.2839
--- Starting trial: run-305
1/1 [==============================] - 0s 1ms/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-306
1/1 [==============================] - 0s 1ms/step - loss: 374.6284 - mean_squared_error: 374.6284
--- Starting trial: run-307
1/1 [==============================] - 0s 859us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-308
1/1 [==============================] - 0s 2ms/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-309
1/1 [==============================] - 0s 2ms/step - loss: 361.9610 - mean_squared_error: 361.9

1/1 [==============================] - 0s 2ms/step - loss: 376.3706 - mean_squared_error: 376.3706
--- Starting trial: run-313
1/1 [==============================] - 0s 1ms/step - loss: 10710.6738 - mean_squared_error: 10710.6738
--- Starting trial: run-314
1/1 [==============================] - 0s 1ms/step - loss: 358.3176 - mean_squared_error: 358.3176
--- Starting trial: run-315
1/1 [==============================] - 0s 1ms/step - loss: 358.3622 - mean_squared_error: 358.3622
--- Starting trial: run-316
1/1 [==============================] - 0s 1ms/step - loss: 363.4520 - mean_squared_error: 363.4520
--- Starting trial: run-317
1/1 [==============================] - 0s 953us/step - loss: 362.8435 - mean_squared_error: 362.8435
--- Starting trial: run-318
1/1 [==============================] - 0s 763us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-319
1/1 [==============================] - 0s 1ms/step - loss: 352.5528 - mean_squared_error: 352.5528


In [14]:
!kill 30958
%tensorboard --logdir logs/hparam_tuning